In [1]:
import pandas as pd
import regex as re
from collections import Counter
import json
import time
import requests
import collections
import matplotlib.pyplot as plt
import numpy as np
import pprint
import pycountry 
import pycountry_convert as pc


## 1. Export patents-pub connections


In [ ]:
# open the raw publication data again so that I can edit it
with open('PATH\data\full-data_scholarly-lens_Genbank[OBIS_and_patents].json') as json_string:
    data_dict = json.load(json_string)
df = pd.read_csv('PATH\data\merged-marine_sequence_data[OBIS_and_patents].csv').drop(columns = ['Unnamed: 0.1'])


In [ ]:
#to check whether I am not getting wrong pubmeds in case The Lens found more than one pubmed connected to the input pubmed, as this may result in biased links
l2 = list(set(df['Pubmed'][~df['Pubmed'].isnull()]))

# publications
patent_ids = []
publications_ids = []
pubmed_ids = []

for i in data_dict: 
    for k, v in i.items(): 
        if k == 'data': 
            for publication in v: 
                for key, value in publication.items(): 
                    if key == 'patent_citations': 
                        for ii in value: 
                            for keyss, valuess in ii.items(): 
                                patent_ids.append(valuess)
                                publications_ids.append(publication['lens_id'])
                                nr = 0
                                for e in publication['external_ids']:                
                                    if 'pmid' in e.values() and int(e['value']) in l2:
                                        nr = nr+1
                                        if nr >= 2:
                                            break
                                        pubmed_ids.append(int(e['value']))

# merge the lists into dict and to pandas                                    
dictdata = {'publication_ids': publications_ids, 'patentID': patent_ids, 'pubmed_IDs': pubmed_ids} # maybe I have to write it as dict instead of dictdata? 
df_patent = pd.DataFrame(dictdata)
           
## export the data files
df_patent.to_csv('PATH\data\pub_patent_edges_Genbank[OBIS_and_patents].csv',index=False) #missing all nan origins now!


In [ ]:
len(set(df_patent['pubmed_IDs'])) #nr of publications references in patents


In [ ]:
len(set(df_patent['patentID'])) 


## 2. Download and export the raw patent data

In [ ]:
#df_patent = pd.read_csv('PATH\data\pub_patent_edges_Genbank[OBIS_and_patents].csv', dtype = 'string')


In [ ]:
# make list of patentIDs (got to include the double quotes); delete duplicates
data = json.dumps(list(set(df_patent['patentID'])))

len(list(set(df_patent['patentID'])))


In [ ]:
data_dict = []
url = 'https://api.lens.org/patent/search'

# include fields
include = '''["doc_key","biblio.parties", "biblio.invention_title", "biblio.references_cited", "biblio.cited_by", "jurisdiction", "date_published", "publication_type"]'''


# request body with scroll time of 1 minute
request_body = '''{
  "query": {
      "terms":  {
          "lens_id": %s
      }
  },
  "include": %s
  , "size": 100
  , "scroll": "1m"
}''' % (data, include)
headers = {'Authorization': 'API_KEY', 'Content-Type': 'application/json'}



# Recursive function to scroll through paginated results
def scroll(scroll_id):
  # Change the request_body to prepare for next scroll api call
  # Make sure to append the include fields to make faster response
    
    if scroll_id is not None:        
        global request_body
        request_body = '''{"scroll_id": "%s", "include": %s}''' % (scroll_id, include)
        
    # make api request
    response = requests.post(url, data=request_body, headers=headers) 
    
    # If rate-limited, wait for n seconds and proceed the same scroll id
    # Since scroll time is 1 minutes, it will give sufficient time to wait and proceed
    if response.status_code == requests.codes.too_many_requests:
        time.sleep(8)
        scroll(scroll_id)
    # If the response is not ok here, better to stop here and debug it
    
    elif response.status_code != requests.codes.ok:
        print(response.json())
        print('Äahh')
    # If the response is ok, do something with the response, take the new scroll id and iterate
    
    # leave the function if it returns empty (because then everything is downloaded already)
    json = response.json()
    if json['data'] == []:
        return
    
    else:
        print('next scroll')
        json = response.json()
        scroll_id = json['scroll_id'] # Extract the new scroll id from response
        #print(json['data']) #DO something with your data
        data_dict.append(response.json())
        
        scroll(scroll_id)
        
# start recursive scrolling
scroll(scroll_id=None)

In [ ]:
#compare list of stored ids with pervious list 
d = []
for n in data_dict: 
    #d = d+len(n['data'])
    for i in n['data']:
        d.append(i['lens_id'])
        
for i in list(set(df_patent['patentID'])):
    if i not in d: 
        print(i)

In [ ]:
# save it
with open('PATH\data\full-data_patent-lens_Genbank[OBIS_and_patents].json', 'w') as f:
    json.dump(data_dict, f)

## 3. Format the patent data

In [3]:
# open the raw patent data again so that I can edit it
with open('PATH\data\full-data_patent-lens_Genbank[OBIS_and_patents].json') as json_string:
    data_dict = json.load(json_string)

In [5]:
patent_id = []
company_countries = []
nr_pat_citations = []
patent_types = []
company_name = []
for batch in data_dict:
    for patent in batch['data']:
        if patent['biblio']['parties']:
            if patent['publication_type'] == 'GRANTED_PATENT' and 'owners_all' in list(patent['biblio']['parties'].keys()):
                for owner in patent['biblio']['parties']['owners_all']:                    
                    company_name.append(owner['extracted_name']['value'])
                    patent_id.append(patent['lens_id'])
                    patent_type = patent['publication_type']
                    patent_types.append(patent_type)
                    if 'patent_count' in patent['biblio']['cited_by']:
                        nr_pat_citations.append(patent['biblio']['cited_by']['patent_count']) 
                    else:
                        nr_pat_citations.append(0)
                    if 'extracted_country' in owner.keys():   
                        company_countries.append(owner['extracted_country'])
                    else:
                        company_countries.append('')
                                                       
            elif patent['publication_type'] == 'PATENT_APPLICATION' and 'applicants' in list(patent['biblio']['parties'].keys()):
                for applicant in patent['biblio']['parties']['applicants']:                    
                    company_name.append(applicant['extracted_name']['value'])
                    patent_id.append(patent['lens_id'])
                    patent_type = patent['publication_type']
                    patent_types.append(patent_type)
                    if 'patent_count' in patent['biblio']['cited_by']:
                        nr_pat_citations.append(patent['biblio']['cited_by']['patent_count']) 
                    else:
                        nr_pat_citations.append(0)
                    if 'residence' in applicant.keys():   
                        company_countries.append(applicant['residence'])
                    else:
                        company_countries.append('')
            else:
                pass
                
                


patents_dict = {'company_country': company_countries, 'patent_id' : patent_id, 'patent_type': patent_types, 'patent_citations' : nr_pat_citations, 'company_name' : company_name}

df_patents = pd.DataFrame(patents_dict)

In [6]:
len(set(df_patents['patent_id'][df_patents['patent_type'] == 'GRANTED_PATENT']))

7170

In [7]:
len(set(df_patents['patent_id'][df_patents['patent_type'] == 'PATENT_APPLICATION']))

5243

## 4. Edit company names   

In [33]:
Counter(df_patents['edited_company_name'][df_patents['edited_company_name'].isin(unedited_names_owners)]).most_common()

[('ZHU QUINN QUN', 5),
 ('NESTEC S.A', 5),
 ('UNIV NEWCASTLE', 5),
 ('BOARD OF REGENTS OF THE UNIVERSITY OF NEBRASKA', 5),
 ('UNIV NAT CHONNAM IND FOUND', 5),
 ('PREVIWO AS', 5),
 ('REPUBLIC KOREA NAT FISHERIES RES & DEVELOPMENT INST', 5),
 ('CSL BEHRING AG', 5),
 ('LIPOTEC SA', 5),
 ('UNIV JIMEI', 5),
 ('UNIV HAINAN', 5),
 ('NAT UNIV GYEONGSANG IACF', 5),
 ('OXITEC LIMITED', 5),
 ('EBNER REINHARD', 5),
 ('NAT INST OF ADV IND & TECHNOL', 5),
 ('UNIV MADRID COMPLUTENSE', 5),
 ('THE UAB RESEARCH FOUNDATION', 5),
 ('GENERAL ATOMICS', 5),
 ('UNIV NANJING', 5),
 ('CURAGEN CORPORATION', 5),
 ('FORSYTH DENTAL INFIRMARY FOR CHILDREN', 5),
 ('SLOAN-KETTERING INSTITUTE FOR CANCER RESEARCH', 5),
 ('FOOD INDUSTRY RESEARCH AND DEVELOPMENT INSTITUTE', 5),
 ('LANXESS CORPORATION', 5),
 ('BRANDEIS UNIVERSITY', 5),
 ('APT KIRK E', 5),
 ('UNIV WAGENINGEN', 5),
 ('PHARMAQ AS', 5),
 ('THE UNITED STATES OF AMERICA AS REPRESENTED BY THE SECRETARY OF AGRICULTURE',
  5),
 ('UCL BUSINESS PLC', 5),
 ('GOLI SURY

In [34]:
for i in df_patents['edited_company_name']:
    if 'CAMBRIA' in i: 
        print(i)


CAMBRIAN INNOVATION
CAMBRIAN INNOVATION
CAMBRIAN INNOVATION
CAMBRIAN INNOVATION
CAMBRIAN INNOVATION
CAMBRIAN INNOVATION
CAMBRIAN INNOVATION
CAMBRIAN INNOVATION


In [35]:
for i,v in owner_country_dict.items(): 
    if 'VERENIUM' in i:
        print(v)
        print(i)
        

US
VERENIUM


In [36]:
for k,v in add_affiliation_dict.items(): 
    if 'CAMBRIAN INNOVATION'  in k:
        print(v)
        print(k)
        

US
CAMBRIAN INNOVATION


In [12]:
# these companies have been identified if they appear more than 5 times (i.e. have more than 5 patents, granted or applied, even before standardization)

edited_names_owners = [ ]
unedited_names_owners = []

for i in df_patents['company_name']: 
    if 'MAO YUMIN' in i or 'PONTEN FREDRIK' in i or 'CRAWFORD JOHN STUART' in i or 'SAYANOVA OLGA' in i or 'SOMANCHI ARAVIND' in i or 'RUDENKO GEORGE' in i or 'NONAKA GEN' in i or 'BETZIG ROBERT ERIC' in i or 'MUELLER WERNER E G' in i or 'MARKOVA SVETLANA' in i or 'FRANK LUDMILA' in i or 'VYSOTSKI EUGENE' in i or 'HILLMAN JENNIFER' in i or 'ERRICO JOSEPH P' in i or 'NIELSEN PREBEN' in i or 'SIMPSON' in i or 'KATO IKUNOSHIN' in i or 'UHLEN MATHIAS' in i or 'OSTERTAG ERIC' in i or 'FRANKLIN SCOTT' in i or 'YAMAMOTO TAKESHI' in i or 'IGAWA TOMOYUKI' in i or 'EINZIGER MICHAEL' in i or 'SMITH JEFFREY' in i or 'HESS HARALD' in i or 'GOLZ STEFAN' in i or 'ROSEN CRAIG' in i or 'JIRSTROEM KARIN' in i or 'TOYO BOSEKI' in i or 'XIE YI' in i or 'RUBEN STEVEN M' in i or 'BAUER JOERG' in i or 'MIYAWAKI ATSUSHI' in i: 
        edited_names_owners.append('individuals')
    elif '1149336' in i or '1149336' in i or '9668241' in i: 
        edited_names_owners.append('to_del')
    elif 'UBIOME' in i or 'PHAIRSON MEDICAL' in i or 'CAMBRIAN INNOVATION' in i or 'MESOBLAST' in i or 'UNIVERSITY TECHNOLOGIES INTERNATIONAL' in i or 'BIO-THERA SOLUTIONS' in i or 'UNITED PET' in i or 'SYNTIRON' in i or 'UNIVERSITY HEALTH NETWORK' in i or 'MED & BIOLOGICAL LAB' in i or 'FORSCHUNGSZENTRUM JUELICH' in i or 'PAIK KWANG' in i or 'HESTZIG' in i or 'ADALTA' in i or 'KONAN GAKUEN' in i or 'NANOTECMARIN' in i or 'CELEXION' in i or 'ONO PHARMACEUTICAL' in i or 'IMPERIAL INNOVATIONS' in i or 'MEDIGENE IMMUNOTHERAPIES' in i or 'PROLUME' in i or 'DIAZYME LABORATORIES' in i or 'ONCURIOUS' in i or 'DIMERIX BIOSCIENCE' in i or 'REALTIME LABORATORIES' in i or 'MYCODART' in i or 'NUTECH' in i or 'GEVO' in i or 'BRAEMAR' in i or 'CROOKEDHOLM' in i or 'KLING' in i or 'PTC THERAPEUTICS' in i or 'GENE STREAM' in i or 'GAUSSIA' in i or 'CERO THERAPEUTICS' in i or 'POLYMARIS' in i or 'KALYPTO' in i or 'GLYCOSYN' in i or 'CELMATIX' in i or 'DEVELOPMENT SWITZERLAND' in i or 'US HEALTH' in i or 'SUN YAT SEN' in i or 'HESKA' in i or 'EPIGENOMICS' in i or 'TARGETED GROWTH' in i or 'MEDICAL & BIOLOGICAL LABORATORIES' in i or 'ASTERIAS BIOTHERAPEUITCS' in i or "CHILDREN'S MEDICAL CENTER CORPORATION" in i or  'GREENLIGHT BIOSCIENCES' in i or 'ARISAPH' in i or 'ENTERIS BIOPHARMA' in i or 'AQUARIA' in i or 'PROTERRO' in i or 'COGNETIX' in i or 'CHEMOURS COMPANY' in i or 'RGH LLC' in i or 'DIADEXUS' in i or 'CYTOMX' in i or 'LUMEN' in i or 'CARIBOU BIOSCIENCES' in i or 'LAWRENCE LIVERMORE NATIONAL SECURITY' in i or 'ONCOTHERAPY' in i or 'RUBIUS' in i or 'SEEGENE' in i or 'FERMENTALG' in i or 'ALPINE' in i or 'BIORIGINAL' in i or 'OSSIANIX' in i or 'MATRIX GENETICS' in i or 'COMPUGEN' in i or 'LUCA TECHNOLOGIES' in i or 'JAPAN TOBACCO' in i or 'DIVERGENCE' in i or 'ALETHIA' in i or 'NATIVE MICROBIALS' in i or 'TWISTDX' in i or 'IDEXX' in i or 'SYNGENTA PARTICIPATIONS' in i or 'AGRIVIDA' in i or 'INTARCIA THERAPEUTICS' in i or 'RAINDANCE TECHNOLOGIES' in i or 'ELWHA' in i or 'PAIRWISE PLANTS SERVICES' in i or 'SAMANA BUSINESS' in i or '10X' in i or 'IMMUNOGEN' in i or 'VIB VZW' in i or 'ATLAS ANTIBODIES' in i or 'GRAINS RESEARCH AND DEVELOPMENT CORPORATION' in i or 'CERES INC' in i or 'ARMISTICE' in i or 'EVELO BIOSCIENCES' in i or 'ICEBERG LUXEMBOURG' in i or 'ACADEMIA SINICA' in i or 'VIENNA' in i or 'RIKEN' in i or 'GUARDANT HEALTH' in i: 
        edited_names_owners.append(i)
    elif 'ALBERT EINSTEIN COLLEGE OF MEDICINE' in i:
        edited_names_owners.append('ALBERT EINSTEIN COLLEGE OF MEDICINE')
    elif 'EVOLVA' in i: 
        edited_names_owners.append('EVOLVA')
    elif 'BIO' in i and 'TECHNE' in i or 'ASURAGEN' in i:
        edited_names_owners.append('BIO-TECHNE')
    elif 'BETH ISRAEL' in i: 
        edited_names_owners.append('BETH ISRAEL DEACONESS MEDICAL CENTER')
    elif 'TOYAMA' in i and 'KYO' not in i: 
        edited_names_owners.append('TOYAMA PREFECTURE')
    elif 'OHIO' in i and 'STATE' in i and 'PHYCO' not in i and 'OIL' not in i: 
        edited_names_owners.append('OHIO STATE UNIVERSITY')
    elif 'OHIO' in i and 'UNIV' in i and 'STATE' not in i: 
        edited_names_owners.append('OHIO UNIVERSITY')
    elif 'NEC PLATFORMS' in i or 'NEC S' in i: 
        edited_names_owners.append('NEC')
    elif 'CALIPER' in i: 
        edited_names_owners.append('CALIPER CORPORATION')
    elif 'YISSUM' in i: 
        edited_names_owners.append('HEBREW UNIVERSITY OF JERUSALEM')
    elif 'HELSINKI' in i and 'UNIV' in i: 
        edited_names_owners.append('HELSINKI UNIVERSITY')
    elif 'SASKATCHEWAN' in i: 
        edited_names_owners.append('UNIVERSITY OF SASKATCHEWAN')
    elif 'MARINE BIOLOGICAL LABORATORY' in i:
        edited_names_owners.append('MARINE BIOLOGICAL LABORATORY')
    elif 'CALI' in i and 'SOUTHERN' in i:
        edited_names_owners.append('UNIVERSITY OF SOUTHERN CALIFORNIA')
    elif 'ZOETIS' in i: 
        edited_names_owners.append('ZOETIS')
    elif 'DALIAN' in i and 'UNIV' in i: 
        edited_names_owners.append('DALIAN OCEAN UNIVERSITY')
    elif 'KYOWA HAKKO' in i or 'KIRIN' in i: 
        edited_names_owners.append('KIRIN')
    elif 'NAGOYA' in i and 'UNIV' in i: 
        edited_names_owners.append('NAGOYA UNIVERSITY')
    elif 'KYOTO' in i and 'UNIV' in i: 
        edited_names_owners.append('UNIVERSITY OF KYOTO')
    elif 'RIGEL' in i: 
        edited_names_owners.append('RIGEL PHARMACEUTICALS')
    elif 'ARCTICZYMES' in i or 'PHARMACON' in i: 
        edited_names_owners.append('ARCTICZYMES')
    elif 'AARHUS' in i: 
        edited_names_owners.append('AARHUS UNIVERSITY')
    elif 'PEBBLE LABS' in i: 
        edited_names_owners.append('PEBBLE LABS')
    elif 'CARNEGIE' in i and 'WASH' in i: 
        edited_names_owners.append('CARNEGIE INSTITUTE OF WASHINGTON')
    elif 'ETH' in i and 'ZUERICH' in i or 'ZÜRICH' in i: 
        edited_names_owners.append('ETH ZUERICH')
    elif 'ALGENOL' in i: 
        edited_names_owners.append('ALGENOL')
    elif 'SHANDONG' in i and 'UNIV' in i:
        edited_names_owners.append('SHANDONG UNIVERSITY')
    elif 'GINKGO' in i or 'GEN9' in i or 'ZYMERGEN' in i:  
        edited_names_owners.append('GINKGO BIOWORKS')
    elif 'STRASBOURG' in i and 'UNIV' in i: 
        edited_names_owners.append('UNIVERSITY OF STRASBOURG')
    elif 'ABGENOMICS' in i or 'ALTRUBIO' in i: 
        edited_names_owners.append('ALTRUBIO')
    elif 'EMERGENT' in i: 
        edited_names_owners.append('EMERGENT BIOSOLUTIONS')
    elif 'GRIFOLS' in i: 
        edited_names_owners.append('GRIFOLS')
    elif 'TUMOR BIOLOGY INVESTMENT GROUP' in i: 
        edited_names_owners.append('TUMOR BIOLOGY INVESTMENT GROUP')
    elif 'YEDA' in i: 
        edited_names_owners.append('WEIZMANN INSTITUTE OF SCIENCE')
    elif 'OXYRANE' in i: 
        edited_names_owners.append('OXYRANE')
    elif 'INVENTION SCIENCE FUND' in i: 
        edited_names_owners.append('INVENTION OF SCIENCE FUND')
    elif 'GRONINGEN' in i and 'UNIV' in i:
        edited_names_owners.append('GRONINGEN UNIVERSITY')
    elif 'AQUAGEN' in i or 'EPITOPIX' in i or 'VAXXINOVA' in i: 
        edited_names_owners.append('EW GROUP')
    elif 'KEYGENE' in i: 
        edited_names_owners.append('KEYGENE')
    elif 'NANYANG' in i: 
        edited_names_owners.append('NANYANG TECHNOLOY UNIVERSITY')
    elif 'RECOMBINANT BIOCATALYSIS' in i: 
        edited_names_owners.append('RECOMBINANT BIOCATALYSIS')
    elif 'OMEROS' in i: 
        edited_names_owners.append('OMEROS CORPORATION')
    elif 'TOKYO' in i and 'INST' in i and 'TECH' in i: 
        edited_names_owners.append('TOKYO INSTITUTE OF TECHNOLOGY')
    elif 'PFIZER' in i or 'ARENA ' in i:
        edited_names_owners.append('PFIZER')
    elif 'SEAGEN' in i or 'SEATTLE GENETICS' in i: 
        edited_names_owners.append('SEAGEN')
    elif 'ROTHAMSTED' in i: 
        edited_names_owners.append('ROTHAMSTED RESEARCH')
    elif 'GOEMAR' in i: 
        edited_names_owners.append('GEOMAR')
    elif 'UNIV NINGBO' in i: 
        edited_names_owners.append('NINGBO UNIVERSITY')
    elif 'CELLTECH' in i: 
        edited_names_owners.append('CELLTECH')
    elif 'SUNTORY' in i: 
        edited_names_owners.append('SUNTORY')
    elif 'AIMM THERAPEUTICS' in i: 
        edited_names_owners.append('AIMM THERAPEUTICS')
    elif 'CARNEGIE MELLON' in i: 
        edited_names_owners.append('CARNIGIE MELLON UNIVERSITY')
    elif 'BATTELLE' in i or 'MIDDLELAND' in i:
        edited_names_owners.append('BATTELLE VENTURES')
    elif 'CANON' in i: 
        edited_names_owners.append('CANON')
    elif 'CHARLES RIVER LABORATORIES' in i: 
        edited_names_owners.append('CHARLES RIVER LABORATORIES')
    elif 'BANK OF NEW YORK MELLON' in i: 
        edited_names_owners.append('BANK OF NEW YORK MELLON')
    elif 'OCEAN NUTRITION' in i: 
        edited_names_owners.append('OCEAN NUTRITION CANADA')
    elif 'CENTRE' in i and 'NAT' in i and 'RECH' in i: 
        edited_names_owners.append('CNRS')
    elif 'AGENCY' in i and 'MARINE' in i:
        edited_names_owners.append('JAPAN AGENCY FOR MARINE-EARTH SCIENCE & TECHNOLOGY')
    elif 'BROAD INST' in i: 
        edited_names_owners.append('THE BROAD INSTITUTE')
    elif 'AUSTRIA' in i and 'AIT' in i:
        edited_names_owners.append('AUSTRIAN INSTITUTE OF TECHNOLOGY')
    elif 'RUTGERS' in i: 
        edited_names_owners.append('RUTGERS UNIVERSITY')
    elif 'ERAGEN BIOSCIENCES' in i or 'LUMINEX' in i: 
        edited_names_owners.append('LUMINEX CORPORATION')
    elif 'EVROGEN' in i: 
        edited_names_owners.append('EVROGEN')
    elif 'DOVETAIL' in i: 
        edited_names_owners.append('CANTATA BIO')
    elif 'ICB INT' in i: 
        edited_names_owners.append('ICB INTERNATIONAL')
    elif 'AVIV' in i: 
        edited_names_owners.append('TEL-AVIV UNIVERSITY')
    elif 'NEWSOUTH INNOVATIONS' in i: 
        edited_names_owners.append('NEWSOUTH INNOVATIONS')
    elif 'BAYLOR' in i: 
        edited_names_owners.append('BAYLOR UNIVERSITY')
    elif 'UNIVERSITY OF OTTAWA' in i: 
        edited_names_owners.append('UNIVERSITY OF OTTOWA')
    elif 'CJ ' in i:
        edited_names_owners.append('CJ GROUP')
    elif 'ROCHESTER' in i: 
        edited_names_owners.append('UNIVERSITY OF ROCHESTER')
    elif 'EISAI' in i: 
        edited_names_owners.append('EISAI')
    elif 'HEMLHOLTZ' in i: 
        edited_names_owners.append('HELMHOLTZ CENTER')
    elif 'KYUSHU' in i and 'UNIV' in i: 
        edited_names_owners.append('KYUSHU UNIVERSITY')
    elif 'LIAONING' in i: 
        edited_names_owners.append('LIAONING UNIVERSITY')
    elif 'PHYLOGICA' in i:
        edited_names_owners.append('PHYLOGICA')
    elif 'PATRYS' in i: 
        edited_names_owners.append('PATRYS')
    elif 'CANCER RES' in i and 'TECH' in i:
        edited_names_owners.append('CANCER RESEARCH UK')
    elif 'UNIV MUENCHEN TECH' in i or 'TECH UNIVERSITÄT MÜNCHEN' in i: 
        edited_names_owners.append('TECHNICAL UNIVERSITY OF MUNICH')
    elif 'UNIV MUENCHEN' in i and 'MAXIM' in i: 
        edited_names_owners.append('LUDWIG MAXIMILIAN UNIVERSITY OF MUNICH')
    elif 'WOODS HOLE OCEANOGRAPHIC' in i: 
        edited_names_owners.append('WOODS HOLE OCEANOGRAPHIC INSTITUTION')
    elif 'INTRON BIOTECHNOLOGY' in i:
        edited_names_owners.append('INTRON BIOTECHNOLOGY')
    elif 'VAXIION THERAPEUTICS' in i: 
        edited_names_owners.append('VAXIION THERAPEUTICS')
    elif 'NANTES' in i: 
        edited_names_owners.append('UNIVERSITY OF NANTES')
    elif 'CORNELL' in i and 'NEW YORK HOSPITAL' not in i: 
        edited_names_owners.append('CORNELL UNIVERSITY')
    elif 'JOULE' in i: 
        edited_names_owners.append('JOULE')
    elif 'NANOCYTE' in i: 
        edited_names_owners.append('NANOYCYTE')
    elif 'JANSSEN' in i and 'RICHARD' not in i or 'SCIOS' in i: 
        edited_names_owners.append('JOHNSON & JOHNSON')
    elif 'LEICESTER' in i: 
        edited_names_owners.append('UNIVERSITY OF LEICESTER')
    elif 'ALAMOS' in i or 'TRIAD' in i: 
        edited_names_owners.append('TRIAD NATIONAL SECURITY')
    elif 'TORAY INDUSTRIES' in i: 
        edited_names_owners.append('TORAY INDUSTRIES')
    elif 'INTEGRATED DNA TECH' in i: 
        edited_names_owners.append('DANAHER CORPORATION')
    elif 'GENELUX' in i: 
        edited_names_owners.append('GENELUX')
    elif 'CELLECTIS' in i: 
        edited_names_owners.append('CELLECTIS')
    elif 'ELASMOGEN' in i: 
        edited_names_owners.append('ELASMOGEN')
    elif 'EVOLUTIONARY GENOMICS' in i: 
        edited_names_owners.append('EVOLUTIONARY GENOMICS')
    elif 'METABOLIX' in i or 'YIELD' in i: 
        edited_names_owners.append('YIELD10 BIOSCIENCE')
    elif 'QUARK PHARMA' in i: 
        edited_names_owners.append('QUARK PHARMACEUTICALS')
    elif 'MIYAZAKI' in i: 
        edited_names_owners.append('UNIVERSITY OF MIYAZAKI')
    elif 'VANDERBILT' in i: 
        edited_names_owners.append('UNIVERSITY OF VANDERBILT')
    elif 'JAPAN' in i and 'SCI' and 'TECH' in i and 'MARINE' not in i and 'CHINA' not in i:
        edited_names_owners.append('JAPAN SCIENCE & TECHNOLOGY AGENCY')
    elif 'CALIFORNIA' in i and 'OF TECHNOLOGY' in i:
        edited_names_owners.append('CALIFORNIA INSTITUTE OF TECHNOLOGY')
    elif 'EVONIK' in i or 'DEGUSSA' in i: 
        edited_names_owners.append('EVONIK')
    elif 'ARENA PHAR' in i:
        edited_names_owners.append('ARENA PHARMACEUTICALS')
    elif 'IOWA' in i and 'UNIV' in i and 'STATE' not in i:
        edited_names_owners.append('UNIVERSITY OF IOWA')
    elif 'NEW YORK UNIVERSITY' in i or 'UNIV NEW YORK' in i:
        edited_names_owners.append('NEW YORK UNIVERSITY')
    elif 'NEW YORK' in i and 'UNIV' in i and 'STATE' in i:
        edited_names_owners.append('NEW YORK STATE UNIVERSITY')
    elif 'IOWA' in i and 'UNIV' in i and 'STATE' in i:
        edited_names_owners.append('IOWA STATE UNIVERSITY')
    elif 'SINGAPORE' in i and 'UNIV' in i:
        edited_names_owners.append('NATIONAL UNIVERSITY OF SINGAPORE')
    elif 'XIAMEN' in i and 'UNIV' in i and 'TECH' not in i:
        edited_names_owners.append('XIAMEN UNIVERSITY')
    elif 'MARA RENEWABLES' in i: 
        edited_names_owners.append('MARA RENEWABLES')
    elif 'THIRD WAVE TECH' in i: 
        edited_names_owners.append('THIRD WAVE TECHNOLOGIES')
    elif 'VEGENICS' in i: 
        edited_names_owners.append('VEGENICS')
    elif 'HOPKINS' in i: 
        edited_names_owners.append('JOHN HOPKINS UNIVERSITY')
    elif 'ADC THERAPEUTICS' in i: 
        edited_names_owners.append('ADC THERAPEUTICS')
    elif 'MED' in i and 'RES' in i and 'COUNCIL' in i:
        edited_names_owners.append('MEDICAL RESEARCH COUNCIL')
    elif 'AB ENZYMES' in i: 
        edited_names_owners.append('AB ENZYMES')
    elif 'ENZYMATICA' in i or 'ZYMETECH' in i: 
        edited_names_owners.append('ENZYMATICA')
    elif 'WACKER CHEMIE' in i: 
        edited_names_owners.append('WACKER CHEMIE')
    elif 'LS9' in i:
        edited_names_owners.append('CHEVRON')
    elif 'NIPPON SUISAN KAISHA' in i:
        edited_names_owners.append('NISSUI')
    elif 'NIPPON' in i and 'OIL' in i:
        edited_names_owners.append('NIPPON OIL & ENERGY CORPORATION')
    elif 'UNITED KINGDOM RESEARCH' in i or 'INNOVATION UK' in i:
        edited_names_owners.append('UK RESEARCH & INNOVATION')
    elif 'MED' in i and 'RES' in i and 'COUNCIL' in i:
        edited_names_owners.append('MEDICAL RESEARCH COUNCIL')
    elif 'GENEOHM' in i: 
        edited_names_owners.append('GENEOHM SCIENCES')
    elif ' GENT' in i and 'UNIV' in i:
        edited_names_owners.append('GENT UNIVERSITY')
    elif 'LABORATORY CORPORATION' in i or 'SEQUENOM' in i: 
        edited_names_owners.append('LABCORP')
    elif 'AGRICULTURE' in i and 'ALIMENTATION' in i or 'RECHERCHE AGRONOMIQUE' in i or 'AGRONOMIQUE INST' in i:
        edited_names_owners.append('INRAE')
    elif 'PSOMAGEN' in i: 
        edited_names_owners.append('MACROGEN')
    elif 'BAYER' in i or 'MONSANTO' in i or 'CALGENE' in i: 
        edited_names_owners.append('BAYER')
    elif 'HUTCHINSON' in i and 'FRED' in i: 
        edited_names_owners.append('FRED HUTCHINSON CANCER CENTER')
    elif 'MARRONE B' in i: 
        edited_names_owners.append('BIOCERES')
    elif 'ADVATECT' in i: 
        edited_names_owners.append('ADVATECT DIAGNOSTICS')
    elif 'KOREA' in i and 'UNIV' in i and 'CATHOLIC' not in i:
        edited_names_owners.append('KOREA UNIVERSITY')
    elif 'BASF' in i:
        edited_names_owners.append('BASF')
    elif 'BIOVERATIV' in i: 
        edited_names_owners.append('BIOVERATIV')
    elif 'ABERDEEN' in i: 
        edited_names_owners.append('UNIVERSITY OF ABERDEEN')
    elif 'FRAUNHOFER' in i:
        edited_names_owners.append('FRAUENHOFER SOCIETY')
    elif 'AZABU V' in i: 
        edited_names_owners.append('AZABU UNIVERSITY')
    elif 'ATOMIQUE' in i:
        edited_names_owners.append('CEA')
    elif 'CONSEJO SUPERIOR' in i: 
        edited_names_owners.append('CONSEJO SUPERIOR INVESTIGACION')
    elif 'APPLIED BIOSYSTEMS' in i or 'AFFYMETRIX' in i or 'THERMO F' in i or 'INTEGENX' in i:
        edited_names_owners.append('THERMO FISHER SCIENTIFIC')
    elif 'UNIV JIANGNAN' in i: 
        edited_names_owners.append('JIANGNAN UNIVERSITY')
    elif 'QIAGEN' in i: 
        edited_names_owners.append('QIAGEN')
    elif 'BGI' in i: 
        edited_names_owners.append('BGI GROUP')
    elif 'KOREA' in i and 'OCEAN' in i:
        edited_names_owners.append('KOREA INSTITUTE OF OCEAN SCIENCE & TECHNOLOGY')
    elif 'TERRAVIA' in i or 'CORBION' in i or 'SOLAZYME' in i: 
        edited_names_owners.append('CORBION')
    elif 'BIOWINDOW' in i: 
        edited_names_owners.append('SHANGHAI BIOWINDOW GENE DEVEVLOPMENT')
    elif 'ZEALAND PHARMA' in i: 
        edited_names_owners.append('ZEALAND PHARMA')
    elif 'DAESANG' in i: 
        edited_names_owners.append('DAESANG')
    elif 'UNIV' in i and 'CURIE' in i:
        edited_names_owners.append('PIERRE AND MARIE CURIE UNIVERSITY')
    elif 'SINGULAR' in i: 
        edited_names_owners.append('SINGULAR GENOMICS SYSTEMS')
    elif 'IFREMER' in i: 
        edited_names_owners.append('IFREMER')
    elif 'TRANSPOSAGEN' in i or 'LONZA' in i: 
        edited_names_owners.append('LONZA')
    elif 'ALDER' in i and 'BIO' in i: 
        edited_names_owners.append('ALDER BIOPHARMACEUTICALS')
    elif 'SEIKAGAKU' in i: 
        edited_names_owners.append('SEIKAGAKU CORPORATION')
    elif 'COUNCIL' in i and 'CA' in i and 'MEDICAL' not in i:
        edited_names_owners.append('NATIONAL RESEARCH COUNCIL CANADA')
    elif 'SANGAMO' in i: 
        edited_names_owners.append('SANGAMO')
    elif 'SANTIAGO' in i and 'COMPOSTELA' in i:
        edited_names_owners.append('UNIVERSITY OF SANTIAGO DE COMPOSTELA')
    elif 'ADIMAB' in i: 
        edited_names_owners.append('ADIMAB')
    elif 'UNILEVER' in i:
        edited_names_owners.append('UNILEVER')
    elif 'KAO CORP' in i: 
        edited_names_owners.append('KAO CORP')
    elif 'OSAKA' in i and 'OSAKA' in i:
        edited_names_owners.append('OSAKA UNIVERSITY')
    elif 'APPLIED MOLECULAR' in i: 
        edited_names_owners.append('APPLIED MOLECULAR TRANSPORT')
    elif 'NUFARM' in i or 'NUSEED' in i: 
        edited_names_owners.append('NUFARM')
    elif 'PASTEUR' in i and 'INSTITUT' in i: 
        edited_names_owners.append('INSTITUT PASTEUR')
    elif 'CODEXIS' in i: 
        edited_names_owners.append('CODEXIS')
    elif 'ADVANCED IND' in i or 'AIST' in i:
        edited_names_owners.append('AIST')
    elif 'DAIICHI SANKYO' in i or 'DAIICHI SEIYAKU' in i:
        edited_names_owners.append('DAIICHI SANKYO')
    elif 'HALOZYME' in i: 
        edited_names_owners.append('HALOZYME')
    elif 'PROCTER & GAMBLE' in i: 
        edited_names_owners.append('PROCTER & GAMBLE')
    elif 'INSERM' in i or 'SANTE' in i and 'ANIMALE' not in i: 
        edited_names_owners.append('INSERM')
    elif 'AGILENT TECHNOLOGIES' in i or 'STRATAGENE' in i or 'POPULATION GENETICS' in i: 
        edited_names_owners.append('AGILENT TECHNOLOGIES')
    elif 'CHUGAI' in i: 
        edited_names_owners.append('CHUGAI PHARMACEUTICAL')
    elif 'AJINOMOTO' in i: 
        edited_names_owners.append('AJINOMOTO')
    elif 'SNIPR' in i: 
        edited_names_owners.append('SNIPR BIOME')
    elif 'INDIGO AG' in i: 
        edited_names_owners.append('INDIGO AG')
    elif 'NOVOZYMES' in i:
        edited_names_owners.append('NOVOZYMES')
    elif 'PURDUE' in i: 
        edited_names_owners.append('PURDUE UNIVERSITY')
    elif 'CURNA' in i or 'OPKO' in i:
        edited_names_owners.append('OPKO')
    elif 'AMGEN' in i or 'TENEOBIO' in i: 
        edited_names_owners.append('AMGEN')
    elif 'ILLUMINA' in i:
        edited_names_owners.append('ILLUMINA')
    elif 'CENTRE NAT RECH SCIENT' in i or 'CNRS' in i:
        edited_names_owners.append('CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE')
    elif 'NATIONAL RESEARCH COUNCIL OF CANADA' in i:
        edited_names_owners.append('NATIONAL RESEARCH COUNCIL OF CANADA')
    elif 'NOVO NORDI' in i or 'FORMA ' in i: 
        edited_names_owners.append('NOVO NORDISK')
    elif 'NEW ENGLAND BIOLABS' in i: 
        edited_names_owners.append('NEW ENGLAND BIOLABS')
    elif 'CHINESE' in i and 'ACAD' in i and 'FISHERIES' not in i or 'SOUTH CHINA SEA' in i or 'OCEANOLOGY CAS' in i:
        edited_names_owners.append('CHINESE ACADEMY OF SCIENCES')
    elif 'CHIN' in i and 'MINERAL' in i:
        edited_names_owners.append('CHINA OCEAN MINERAL RESOURCES RESEARCH AND DEVELOPMENT ASSOCIATION')
    elif 'CHINESE' in i and 'ACAD' in i and 'FISHERIES' in i:
        edited_names_owners.append('CHINESE ACADEMY OF FISHERY SCIENCES') 
    elif 'CHIN' in i and 'OCEAN' in i and 'UNIV' in i:
        edited_names_owners.append('OCEAN UNIVERSITY OF CHINA')
    elif 'CALIFORNIA INST OF TECHN' in i:
        edited_names_owners.append('CALIFORNIA INSTITUTE OF TECHNOLOGY')
    elif 'TOKYO' in i and 'UNIV' in i and 'MARINE' in i:
        edited_names_owners.append('TOKYO UNIVERSITY OF MARINE SCIENCE AND TECHNOLOGY')
    elif 'TOKYO' in i and 'UNIV' in i and 'INST' in i:
        edited_names_owners.append('TOKYO INSTITUTE OF TECHNOLOGY')
    elif 'TOKYO' in i and 'UNIV' in i and 'AGRI' in i:
        edited_names_owners.append('TOKYO UNIVERSITY OF AGRICULTURE AND TECHNOLOGY')
    elif 'TOKYO' in i and 'UNIV' in i and 'TECH' not in i and 'MED' not in i and 'MARINE' not in i and 'PHARMACY' not in i:
        edited_names_owners.append('UNIVERSITY OF TOKYO')
    elif 'UNIVERSITY OF CALIFORNIA' in i or 'UNIV CALIFORNIA' in i:
        edited_names_owners.append('UNIVERSITY OF CALIFORNIA')
    elif 'PIONEER HI' in i or 'CORTEVA' in i:
        edited_names_owners.append('CORTEVA')
    elif 'DOW AGROSCIENCES' in i or 'ROHM' in i: 
        edited_names_owners.append('DOW')
    elif 'GENOMATICA' in i or 'REG LIFE SCIENCES' in i: 
        edited_names_owners.append('GENOMATICA')
    elif 'COMM' in i and 'ORG' in i: 
        edited_names_owners.append('CSIRO')
    elif 'FINCH THERAPEUTICS' in i: 
        edited_names_owners.append('FINCH THERAPEUTICS')
    elif 'MILLENIUM' in i: 
        edited_names_owners.append('MILLENIUM PHARMACEUTICALS')
    elif 'ASTRA' in i or 'ZENECA' in i or 'MEDIMMUNE' in i or 'SPIROGEN' in i or 'ONCOMAB' in i:
        edited_names_owners.append('ASTRAZENECA')
    elif 'HUMAN GENOME SCIENCE' in i or 'GLAXO' in i or 'SMITHKLINE' in i or 'DOMANTIS' in i: 
        edited_names_owners.append('GLAXOSMITHKLINE')
    elif '4D PHARMA' in i:
        edited_names_owners.append('4D PHARMA')
    elif 'DSM' in i:
        edited_names_owners.append('DSM')
    elif 'DNAE' in i or 'DNA ELE' in i: 
        edited_names_owners.append('DNAe')
    elif 'HARVARD COLLEGE' in i or 'FARBER' in i:
        edited_names_owners.append('HARVARD UNIVERSITY')
    elif 'STANFORD JUNIOR' in i:
        edited_names_owners.append('STANFORD UNIVERSITY')
    elif 'HENKEL' in i:
        edited_names_owners.append('HENKEL')
    elif 'DOW GLOBAL TECHNOLOGIES' in i or 'ROHM AND HAAS COMPANY' in i:
        edited_names_owners.append('DOW GLOBAL TECHNOLOGIES')
    elif 'DU PONT' in i or 'DUPONT' in i or 'DANISCO' in i or 'TAXON BIOSCIENCE' in i:
        edited_names_owners.append('DU PONT')
    elif 'NUVELO' in i: 
        edited_names_owners.append('ARCA BIOPHARMA')
    elif 'AMRYT' in i or 'CHIASMA' in i:
        edited_names_owners.append('AMRYT')
    elif 'MARTEK BIOSCIENCES' in i:
        edited_names_owners.append('MARTEK BIOSCIENCES')
    elif 'AMARIN PHARMA' in i:
        edited_names_owners.append('AMARIN PHARMA')
    elif 'MERCK' in i or 'SERONO' in i or 'INTERVET' in i or 'MSD' in i or 'HARRISVACCINES' in i:
        edited_names_owners.append('MERCK')
    elif 'HOWARD HUGHES' in i: 
        edited_names_owners.append('HOWARD HUGHES MEDICAL INSTITUTE')
    elif 'PROMEGA CORP' in i: 
        edited_names_owners.append('PROMEGA')
    elif 'PLANCK' in i: 
        edited_names_owners.append('MAX-PLANCK-GESELLSCHAFT')
    elif 'BECTON DICKINSON AND CO' in i or 'CELLULAR RESEARCH' in i: 
        edited_names_owners.append('BECTON DICKINSON AND COMPANY')
    elif 'WISCONSIN ALUMNI' in i: 
        edited_names_owners.append('WISCONSIN ALUMNI RESEARCH FOUNDATION')
    elif 'SMITH & NEPHEW' in i or 'OSIRIS' in i: 
        edited_names_owners.append('SMITH & NEPHEW')
    elif 'VERENIUM' in i: 
        edited_names_owners.append('VERENIUM')
    elif 'SMITH' in i and 'NEPHEW' in i: 
        edited_names_owners.append('SMITH & NEPHEW')
    elif 'LIFE TECHNOLOGIES' in i: 
        edited_names_owners.append('LIFE TECHNOLOGIES')
    elif 'GEN' in i and 'HOSPITAL' in i and 'CORP' in i: 
        edited_names_owners.append('THE GENERAL HOSPITAL CORPORATION')
    elif 'GEN-PROBE' in i or 'HOLOGIC' in i: 
        edited_names_owners.append('HOLOGIC')
    elif 'MASSACHUSETTS INST' in i: 
        edited_names_owners.append('MASSACHUSETTS INSTITUTE OF TECHNOLOGY')
    elif 'CAROLINA STATE' in i: 
        edited_names_owners.append('NORTH CAROLINA STATE UNIVERSITY')
    elif 'NORTH CAROLINA' in i and 'STATE' not in i: 
        edited_names_owners.append('UNIVERSITY OF NORTH CAROLINA')
    elif 'JNC CORP' in i or 'CHISSO' in i: 
        edited_names_owners.append('JNC CORPORATION')
    elif 'ABBVIE' in i or 'STEMCENTRX' in i or 'ALLERGAN' in i: 
        edited_names_owners.append('ABBVIE')
    elif 'INCYTE' in i: 
        edited_names_owners.append('INCYTE')
    elif 'YALE' in i: 
        edited_names_owners.append('YALE UNIVERSITY')
    elif 'GERON C' in i: 
        edited_names_owners.append('GERON')
    elif 'CRESTOVO' in i: 
        edited_names_owners.append('CRESTOVO')
    elif 'COLUMBIA UNIVERSITY' in i or 'UNIV COLUMBIA' in i and 'BRITISH' not in i: 
        edited_names_owners.append('COLOMBIA UNIVERSITY')
    elif 'COLUMBIA' in i and 'BRITISH' in i and 'STRESSGEN' not in i: 
        edited_names_owners.append('UNIVERSITY OF BRITISH COLUMBIA')
    elif 'INVISTA' in i or 'INV NYLON' in i: 
        edited_names_owners.append('INVISTA')
    elif 'UNIV' in i and 'MICHIGAN' in i and 'STATE' not in i: 
        edited_names_owners.append('UNIVERSITY OF MICHIGAN')
    elif 'UNIV' in i and 'MICHIGAN' in i and 'STATE' in i: 
        edited_names_owners.append('MICHIGAN STATE UNIVERSITY')
    elif 'UNIV' in i and 'ARIZONA' in i and 'STATE' not in i: 
        edited_names_owners.append('ARIZONA STATE UNIVERSITY')
    elif 'UNIV' in i and 'ARIZONA' in i and 'UNIV' in i and 'NORTH' not in i: 
        edited_names_owners.append('UNIVERSITY OF ARIZONA')
    elif 'UNIV' in i and 'ARIZONA' in i and 'UNIV' in i and 'NORTH' in i: 
        edited_names_owners.append('NORTHERN ARIZONA UNIVERSITY')
    elif 'WASHINGTON UNIVERSITY' in i: 
        edited_names_owners.append('WASHINGTON UNIVERSITY')
    elif 'UNIV' in i and 'WASHINGTON' in i and 'WASHINGTON UNIVERSITY' not in i and 'STATE' not in i: 
        edited_names_owners.append('UNIVERSITY OF WASHINGTON')
    elif 'UNIV' in i and 'WASHINGTON' in i and 'WASHINGTON UNIVERSITY' not in i and 'STATE' in i: 
        edited_names_owners.append('WASHINGTON STATE UNIVERSITY')
    elif 'UNIV' in i and 'TEXAS' in i: 
        edited_names_owners.append('UNIERSITY OF TEXAS')
    elif 'DUKE' in i: 
        edited_names_owners.append('DUKE UNIVERSITY')
    elif 'GEORGIA' in i and 'UNIV' in i: 
        edited_names_owners.append('UNIVERSITY OF GEORGIA')
    elif 'SERES' in i: 
        edited_names_owners.append('SERES')
    elif 'UTAH' in i and 'UNIV' in i: 
        edited_names_owners.append('UNIVERSITY OF UTAH')
    elif 'PITTSBURGH' in i: 
        edited_names_owners.append('UNIVERSITY OF PITTSBURGH')
    elif 'UNIV' in i and 'FLORIDA' in i: 
        edited_names_owners.append('UNIVERSITY OF FLORIDA')
    elif 'UNIV' in i and 'SOUTH FLORIDA' in i: 
        edited_names_owners.append('UNIVERSITY OF SOUTH FLORIDA')
    elif 'TAKARA' in i or 'CLONTECH' in i: 
        edited_names_owners.append('TAKARA')
    elif 'BRIGHAM' in i: 
        edited_names_owners.append("BRIGHAM AND WOMEN'S HOSPITAL")
    elif 'UNIV' in i and 'MINNESOTA' in i: 
        edited_names_owners.append('UNIVERSITY OF MINNESOTA')
    elif 'UNIV' in i and 'PENNSYLVANIA' in i: 
        edited_names_owners.append('UNIVERSITY OF PENNSYLVANIA')
    elif 'COLORADO' in i and 'UNIV' in i and 'STATE' not in i: 
        edited_names_owners.append('UNIVERSITY OF COLORADO')
    elif 'COLORADO' in i and 'UNIV' in i and 'STATE' in i: 
        edited_names_owners.append('COLORADO STATE UNIVERSITY')
    elif 'NOVARTIS' in i or 'CHIRON' in i: 
        edited_names_owners.append('NOVARTIS')
    elif 'TRANSWORLD' in i: 
        edited_names_owners.append('TRANSWORLD TECHNOLOGIES')
    elif 'ALEXION' in i: 
        edited_names_owners.append('ALEXION')
    elif 'INVITROGEN' in i: 
        edited_names_owners.append('INVITROGEN')
    elif 'BP COR' in i:  
        edited_names_owners.append('BP')
    elif 'UNIVERSITY LICENSE' in i: 
        edited_names_owners.append('UNIVERSITY LICENSE EQUITY HOLDING')
    elif 'TUFTS' in i or 'NEW ENGLAND MEDICAL CENTER' in i:
        edited_names_owners.append('TUFTS UNIVERSITY')
    elif 'TUFTS' in i and 'COLLEGE' in i: 
        edited_names_owners.append('TUFTS COLLEGE')
    elif 'NORTHWESTERN' in i: 
        edited_names_owners.append('NORTHWESTERN UNIVERSITY')
    elif 'GENENCOR' in i: 
        edited_names_owners.append('GENENCOR')
    elif 'MAYO FOUND' in i: 
        edited_names_owners.append('MAYO FOUNDATION')
    elif 'STRYKER' in i or 'NOVADAQ' in i: 
        edited_names_owners.append('STRYKER')
    elif 'ROSETTA' in i: 
        edited_names_owners.append('ROSETTA INPHARMATICS')
    elif 'GENERAL HOSPITAL CORPORATION' in i: 
        edited_names_owners.append('GENERAL HOSPITAL CORPORATION')
    elif 'ELANCO' in i: 
        edited_names_owners.append('ELANCO')
    elif 'WYETH' in i: 
        edited_names_owners.append('WYETH')
    elif 'U.S. SMOKELESS TOBACCO COMPANY' in i: 
        edited_names_owners.append('U.S. SMOKELESS TOBACCO COMPANY')
    elif 'EXXON' in i: 
        edited_names_owners.append('EXXONMOBIL')
    elif 'SCRIPPS' in i: 
        edited_names_owners.append('SCRIPPS RESEARCH INSTITUTE')
    elif 'KENTUCKY' in i and 'UNIV' in i: 
        edited_names_owners.append('UNIVERSITY OF KENTUCKY')
    elif 'PENN STATE' in i: 
        edited_names_owners.append('PENNSYLVANIA STATE UNIVERSITY')
    elif 'INTREXON' in i or 'RHEOGENE' in i: 
        edited_names_owners.append('INTREXON')
    elif 'BIO RAD' in i or 'BIO-RAD' in i: 
        edited_names_owners.append('BIO-RAD LABORATORIES')
    elif 'LA ROCHE' in i or 'ROCHE ' in i or 'GENENTECH' in i or 'BOEHRINGER MANNHEIM' in i:
        edited_names_owners.append('ROCHE')
    elif 'ISIS ' in i: 
        edited_names_owners.append('ISIS')
    elif 'MARICAL' in i: 
        edited_names_owners.append('MARICAL')
    elif 'PRINCETON' in i and 'UNIV' in i: 
        edited_names_owners.append('PRINCETON UNIVERSITY')
    elif 'ABBOTT' in i or 'IBIS BIOSCIENCES' in i or 'IONIS' in i or 'ALERE' in i: 
        edited_names_owners.append('ABBOTT')        
    elif 'ASTELLAS' in i or 'AGENSYS' in i: 
        edited_names_owners.append('ASTELLAS')     
    elif 'COLD SPRING' in i: 
        edited_names_owners.append('COLD SPRING HARBOR LABORATORY')
    elif 'EMORY' in i and 'UNIV' in i: 
        edited_names_owners.append('EMORY UNIVERSITY')
    elif 'ONCOMED' in i or 'MEREO' in i: 
        edited_names_owners.append('MEREO BIOPHARMA')
    elif 'CEDARS-SINAI MEDICAL' in i: 
        edited_names_owners.append('CEDARS-SINAI MEDICAL CENTER')
    elif 'REGENERON' in i: 
        edited_names_owners.append('REGENERON')
    elif 'UNIVERSITY OF JERUSALEM' in i:
        edited_names_owners.append('UNIVERSITY OF JERUSALEM')
    elif 'SYNTHETIC GENOMICS' in i or 'VIRIDOS' in i: 
        edited_names_owners.append('VIRIDOS')
    elif 'YORKTOWN TECHNOLOGIES' in i or 'GLOFISH' in i: 
        edited_names_owners.append('YORKTOWN TECHNOLOGIES')
    elif 'MIDPOINT FOOD' in i: 
        edited_names_owners.append('MIDPOINT FOOD AND AG CO-INVESTMENT FUND')
    elif 'SANOFI' in i or 'AMUNIX' in i or 'TRANSLATE BIO' in i: 
        edited_names_owners.append('SANOFI')
    elif 'UNIGENE LABORATORIES' in i: 
        edited_names_owners.append('UNIGENE LABORATORIES')
    elif 'INSTITUTE FOR GENOMIC RESEARCH'  in i or 'CRAIG VENTER' in i:
        edited_names_owners.append('J. CRAIG VENTER INSTITUTE')
    elif 'OCERA THERAPEUTICS' in i: 
        edited_names_owners.append('MALLINCKRODT PHARMACEUTICALS')
    elif 'TETRA' in i: 
        edited_names_owners.append('TETRA HOLDING')
    elif 'NEW HAMPSHIRE' in i:
        edited_names_owners.append('UNIVERSITY OF NEW HAMPSHIRE')
    elif 'MARYLAND' in i: 
        edited_names_owners.append('UNIVERSITY OF MARYLAND')
    elif 'CHICAGO' in i and 'UNIV' in i: 
        edited_names_owners.append('UNIVERSITY OF CHICAGO')
    elif 'BIOGEN ' in i and 'MICRO' not in i: 
        edited_names_owners.append('BIOGEN')
    elif 'MYERS ' in i or 'MEDAREX' in i or 'ZYMOGEN' in i or 'SQUIBB' in i: 
        edited_names_owners.append('BRISTOL-MYERS SQUIBB COMPANY')
    elif 'VIRGINIA TECH INTELLECTUAL PROPERTIES' in i:
        edited_names_owners.append('VIRGINIA TECH INTELLECTUAL PROPERTIES')
    elif 'VERTEX PHARMA' in i: 
        edited_names_owners.append('VERTEX PHARMACEUTICALS')
    elif 'BAXTER' in i: 
        edited_names_owners.append('BAXTER')
    elif 'SHIRE-NPS' in i or 'TAKEDA' in i or 'BAXALTA' in i or 'MILLENNIUM PHARMACEUTICALS' in i:
        edited_names_owners.append('TAKEDA')
    elif 'TENNESSE' in i or 'UT-BATTELLE' in i: 
        edited_names_owners.append('UNIVERSITY OF TENNESSEE')
    elif 'INNOVATION VALLEY' in i or 'BATTELLE' in i and 'UT-BATTELLE' not in i: 
        edited_names_owners.append('BATTELLE')
    elif 'XOMA' in i:
        edited_names_owners.append('XOMA')
    elif 'TRANZYME' in i: 
        edited_names_owners.append('TRANZYME')
    elif 'UCB' in i: 
        edited_names_owners.append('UCB')
    else:
        unedited_names_owners.append(i)
        edited_names_owners.append(i)

        

        
# the endings off particular strings
endings_4 = (' INC', ' LTD', ' LLC')
endings_3 = (' LP')
endings_6 = (' L.L.C')
edited_names_owners_noincs = []
for name in edited_names_owners: 
    if name.endswith(endings_4):
        edited_names_owners_noincs.append(name[:-4])
    elif name.endswith(endings_3):
        edited_names_owners_noincs.append(name[:-3])
    elif name.endswith(endings_6):
        edited_names_owners_noincs.append(name[:-6])
    else: 
        edited_names_owners_noincs.append(name)
        
unedited_names_owners_noincs = []
for name in unedited_names_owners: 
    if name.endswith(endings_4):
        unedited_names_owners_noincs.append(name[:-4])
    elif name.endswith(endings_3):
        unedited_names_owners_noincs.append(name[:-3])
    elif name.endswith(endings_6):
        unedited_names_owners_noincs.append(name[:-6])
    else: 
        unedited_names_owners_noincs.append(name)

df_patents['edited_company_name'] = edited_names_owners_noincs



In [13]:
Counter(df_patents['edited_company_name'][df_patents['patent_type'] == 'GRANTED_PATENT']).most_common()

[('UNIVERSITY OF CALIFORNIA', 237),
 ('DU PONT', 212),
 ('BAYER', 170),
 ('HARVARD UNIVERSITY', 159),
 ('DSM', 126),
 ('STANFORD UNIVERSITY', 117),
 ('MARTEK BIOSCIENCES', 112),
 ('SMITH & NEPHEW', 111),
 ('BECTON DICKINSON AND COMPANY', 111),
 ('OPKO', 109),
 ('NOVARTIS', 109),
 ('BASF', 104),
 ('STRYKER', 96),
 ('ABBVIE', 92),
 ('HOWARD HUGHES MEDICAL INSTITUTE', 88),
 ('TAKEDA', 84),
 ('ABBOTT', 83),
 ('MERCK', 83),
 ('ROCHE', 82),
 ('ASTRAZENECA', 78),
 ('UNIVERSITY OF FLORIDA', 77),
 ('INCYTE', 71),
 ('PROMEGA', 70),
 ('AMARIN PHARMA', 70),
 ('GENOMATICA', 69),
 ('CSIRO', 67),
 ('INTREXON', 67),
 ('UNIVERSITY OF UTAH', 66),
 ('MACROGEN', 64),
 ('UBIOME', 63),
 ('THE GENERAL HOSPITAL CORPORATION', 60),
 ('INVISTA', 59),
 ('TAKARA', 57),
 ('4D PHARMA', 56),
 ('GLAXOSMITHKLINE', 55),
 ('CORBION', 54),
 ('ILLUMINA', 53),
 ('UNIVERSITY OF VIENNA', 53),
 ('JNC CORPORATION', 53),
 ('NUFARM', 53),
 ('ARMISTICE CAPITAL MASTER FUND', 52),
 ('NOVOZYMES', 52),
 ('UNIVERSITY OF MARYLAND', 50),

In [42]:
len(set(df_patents['patent_id'][df_patents['patent_type'] == 'GRANTED_PATENT']))

7170

In [14]:
#filter out individual and to_del 
to_delete = ['individuals', 'to_del']
df_patents_ed = df_patents[~df_patents['edited_company_name'].isin(to_delete)]



In [43]:
len(set(df_patents_ed['patent_id'][df_patents['patent_type'] == 'GRANTED_PATENT']))

7150

## 5. Add country column

In [15]:
# try running through all the patents regardless of application or granted patent --> because applications have much better country classifications
owner_country_dict = {}
count = 0

manual_decisions = []
for patent in list(set(df_patents_ed['patent_id'])):
        print(count)
        for owner in df_patents_ed['edited_company_name'][df_patents_ed['patent_id'] == patent]:
            owner_country = list(df_patents_ed['company_country'][(df_patents_ed['patent_id'] == patent) & (df_patents_ed['edited_company_name'] == owner)])[0]
            if owner_country != '':    
                if owner in owner_country_dict.keys(): # in case it is already in there and it contradicts, move to manual decision
                    if owner_country not in owner_country_dict[owner]:
                        manual_decisions.append(owner)
                        owner_country_dict.pop(owner, None)
                        print('deleteting:' + str(owner))
                    else: 
                        print('matching owner countries: '+ str(owner))
                elif owner in manual_decisions: # if already in manual decision, keeep there
                    print('this owner already in manual decisions:' + str(owner))
                    pass 

                else: 
                    owner_country_dict[owner] = owner_country
                    print('added this owner: ' + str(owner) +'and this country: ' + str(owner_country) )
            else: 
                print('no country listed for: ' + str(owner))
                pass    
    
        
        count = count +1
        #if count == 1000: 
        #    break
                
    #elif len(df_patents_ed[df_patents_ed['patent_id'] == patent]) > 1:
        #multiple but different: 
        
    #if df_patents_ed['edited_owner_names'][df_patents_ed['patent_id'] == patent] >= 2: # in case there are multiple owners 
    #    if and 
    

0
added this owner: CANCER RESEARCH UKand this country: GB
no country listed for: YALE UNIVERSITY
1
added this owner: UNIV AUBURNand this country: US
2
no country listed for: OPKO
3
added this owner: CRC FOR WASTE MANAGEMENT AND POLLUTION CONTROL LIMITEDand this country: AU
4
added this owner: OSSIANIXand this country: US
5
no country listed for: GEORGIA TECH RESEARCH CORPORATION
6
added this owner: GSF-FORSCHUNGSZENTRUM FUR UMWELT UND GESUNDHEIT GMBHand this country: DE
7
added this owner: AISTand this country: JP
8
added this owner: PROCTER & GAMBLEand this country: US
9
no country listed for: WYETH
10
added this owner: EVELO BIOSCIENCESand this country: US
11
no country listed for: D.E. SHAW RESEARCH
12
no country listed for: UNIVERSITY OF MIAMI
13
added this owner: S2B GMBH & CO KGand this country: DE
14
added this owner: NATIONAL RESEARCH COUNCIL CANADAand this country: CA
15
added this owner: UNIVERSITY OF ABERDEENand this country: GB
added this owner: 4D PHARMAand this country: 

added this owner: LAFLEUR DAVID Wand this country: US
added this owner: OLSEN HENRIK Sand this country: US
added this owner: SHI YANGGUand this country: US
added this owner: MOORE PAUL Aand this country: US
89
added this owner: AGILENT TECHNOLOGIESand this country: SG
matching owner countries: AGILENT TECHNOLOGIES
matching owner countries: AGILENT TECHNOLOGIES
90
matching owner countries: GLAXOSMITHKLINE
matching owner countries: EBNER REINHARD
91
no country listed for: BATTELLE VENTURES
no country listed for: U.S. DEPARTMENT OF ENERGY
no country listed for: BECHTEL BWXT IDAHO
92
added this owner: MARTEK BIOSCIENCESand this country: US
93
no country listed for: THE GENERAL HOSPITAL CORPORATION
94
added this owner: CANCER REC TECHand this country: GB
95
matching owner countries: ORF LIFTAEKNI HF
96
added this owner: TOYAMA PREFECTUREand this country: JP
matching owner countries: AJINOMOTO
added this owner: ASANO YASUHISAand this country: JP
added this owner: MATSUDA MOTOKIand this count

no country listed for: PEBBLE LABS
178
added this owner: UNIV MCGILLand this country: CA
added this owner: PELLETIER JERRYand this country: CA
added this owner: DAS MANJULAand this country: CA
179
added this owner: KAPLAN LEE Mand this country: US
added this owner: STYLOPOULOS NICHOLASand this country: US
added this owner: HARRIS JASON Land this country: US
added this owner: HENNINGER DWIGHTand this country: US
added this owner: ARONHALT TAYLOR Wand this country: US
added this owner: VOEGELE JAMES Wand this country: US
added this owner: THE GENERAL HOSPITAL CORPORATIONand this country: US
added this owner: ETHICON ENDO SURGERYand this country: US
180
added this owner: ALETHIA BIOTHERAPEUTICSand this country: CA
181
added this owner: PURDUE UNIVERSITYand this country: US
added this owner: GELVIN STANTON Band this country: US
added this owner: SARDESAI NAGESHand this country: US
182
no country listed for: CELLTECH
183
added this owner: JENNEWEIN BIOTECHNOLOGIE GMBHand this country: DE
18

added this owner: ELANGO NAVINand this country: US
282
matching owner countries: BRIGHAM AND WOMEN'S HOSPITAL
283
added this owner: CSIROand this country: AU
284
matching owner countries: EVELO BIOSCIENCES
285
no country listed for: STANFORD UNIVERSITY
286
deleteting:DSM
287
added this owner: CTC BIOand this country: KR
288
added this owner: IFREMERand this country: FR
added this owner: INST DE CANCEROLOGIE DE L’OUESTand this country: FR
added this owner: UNIVERSITY OF NANTESand this country: FR
added this owner: INSERMand this country: FR
289
added this owner: ENSOL BIOSCIENCESand this country: KR
290
no country listed for: エフ．ホフマン−ラ ロシュ アーゲーＦ． ＨＯＦＦＭＡＮＮ−ＬＡ ＲＯＣＨＥ ＡＫＴＩＥＮＧＥＳＥＬＬＳＣＨＡＦＴ
291
no country listed for: THIRD INST OCEANOGRAPHY SOA
292
added this owner: THERMO FISHER SCIENTIFICand this country: US
added this owner: LEMIEUX BERTRANDand this country: CA
added this owner: LANDRY BENOIT Sand this country: CA
added this owner: SAPOLSKY RONALD Jand this country: US
added this owner: MURI

added this owner: NAT MARINE BIODIVERSITY INSTITUTE OF KOREAand this country: KR
385
no country listed for: UNIV ABERYSTWYTH
added this owner: ALLAINGUILLAUME JOELand this country: GB
added this owner: WILKINSON MICHAEL JAMESand this country: GB
386
no country listed for: BAYER
this owner already in manual decisions:DSM
no country listed for: MARTEK BIOSCIENCES
no country listed for: MARTEK BIOSCIENCES
387
added this owner: JAPAN SCIENCE & TECHNOLOGY AGENCYand this country: JP
388
no country listed for: CLEVELAND CLINIC FOUNDATION THE
389
no country listed for: DSM
390
no country listed for: AOBIOME
391
added this owner: BIO-THERA SOLUTIONSand this country: CN
392
added this owner: ACADEMIA SINICAand this country: TW
393
matching owner countries: NOVOZYMES
added this owner: HARRIS PAULand this country: US
added this owner: MAIYURAN SUCHINDRAand this country: US
added this owner: MCBRAYER BRETTand this country: US
added this owner: MUZZI-ERICHSEN GLORIAand this country: US
394
no countr

...

no country listed for: UNIVERSITY OF FLORIDA
11918
no country listed for: TOYOTA CENTRAL RES & DEV
11919
matching owner countries: UNIV SANTIAGO CHILE
11920
matching owner countries: EPIVA BIOSCIENCES
added this owner: RAHMAN SHILAand this country: US
11921
matching owner countries: CANCER RESEARCH UK
11922
matching owner countries: AIST
11923
matching owner countries: MEREO BIOPHARMA
11924
added this owner: SECUTECH INTERNAT PTEand this country: SG
11925
matching owner countries: ALMAC DISCOVERY
11926
this owner already in manual decisions:ASTRAZENECA
11927
added this owner: BIONEER CORPORATIONand this country: KR
11928
this owner already in manual decisions:NOVARTIS
this owner already in manual decisions:NOVARTIS
this owner already in manual decisions:GRIFOLS
11929
no country listed for: UNIVERSITY OF MINNESOTA
11930
no country listed for: JIANGNAN UNIVERSITY
11931
matching owner countries: CELLECTIS
11932
matching owner countries: UNIVERSITY OF CALIFORNIA
11933
matching owner countr

no country listed for: OHIO UNIVERSITY
12018
no country listed for: OPKO
12019
matching owner countries: NAT UNIV GYEONGSANG IACF
12020
this owner already in manual decisions:NOVOZYMES
12021
added this owner: OKINAWA INST OF SCIENCE AND TECHNOLOGY SCHOOL CORPand this country: JP
12022
this owner already in manual decisions:NOVOZYMES
12023
matching owner countries: SHANGHAI BIOWINDOW GENE DEVEVLOPMENT
12024
matching owner countries: DNA GENSEE SAS
12025
added this owner: ASAHI BREWERIESand this country: JP
added this owner: MUKAI YUKOand this country: JP
added this owner: KAWAMOTO YACHIYOand this country: JP
added this owner: YOKOTA TOYOKAZUand this country: JP
added this owner: SAMI MANABUand this country: JP
12026
matching owner countries: HENKEL
12027
added this owner: MARUHA NICHIRO FOODSand this country: JP
12028
no country listed for: BIO-TECHNE
12029
no country listed for: SERES
12030
no country listed for: GUARDANT HEALTH
12031
no country listed for: IMMUGEN PHARMA
12032
matchin

matching owner countries: CNRS
matching owner countries: ECOLE NORM SUPERIEURE LYON
added this owner: PEPSCAN SYSTEMS BVand this country: NL
added this owner: MELY YVESand this country: FR
added this owner: LANGEDIJK JOHANNESand this country: NL
added this owner: SHVADCHAK VOLODYMYRand this country: UA
added this owner: DARLIX JEAN-LUCand this country: FR
12115
matching owner countries: KAO CORP
12116
this owner already in manual decisions:JNC CORPORATION
12117
this owner already in manual decisions:ROCHE
12118
matching owner countries: BIOSOLUM B V
12119
matching owner countries: SEAGEN
12120
no country listed for: ALGENOL
12121
added this owner: HYUNIL-BIO COand this country: KR
12122
this owner already in manual decisions:NOVOZYMES
this owner already in manual decisions:NOVOZYMES
added this owner: MONOSOLand this country: US
12123
no country listed for: ECKENRODE, VIRGINIA K, ALABAMA
12124
no country listed for: UNIVERSITY OF CALIFORNIA
matching owner countries: UNIVERSITY OF VIENNA

no country listed for: UNIV NANJING AGRICULTURAL
12211
this owner already in manual decisions:DSM
this owner already in manual decisions:NOVOZYMES
12212
matching owner countries: FRED HUTCHINSON CANCER CENTER
added this owner: FREDRICKS DAVID Nand this country: US
added this owner: KHOT PRASANNA Dand this country: US
added this owner: KO DAISY Land this country: US
12213
no country listed for: STANFORD UNIVERSITY
12214
no country listed for: COLD SPRING HARBOR LABORATORY
12215
matching owner countries: VACCINEX
12216
added this owner: SUIGENERIS FARM S Land this country: ES
12217
no country listed for: MICHIGAN STATE UNIVERSITY
12218
added this owner: CHINA NATIONAL SEED GROUP CO.and this country: CN
added this owner: HUAZHONG AGRICULTURAL UNIVERSITYand this country: CN
added this owner: SHENZHEN INSTITUTE OF MOLECULAR CROP DESIGNand this country: CN
added this owner: FRONTIER LABORATORIES OF SYSTEMS CROP DESIGN CO.and this country: CN
12219
matching owner countries: PREVIWO AS
12220
n

no country listed for: STANFORD UNIVERSITY
12306
added this owner: MERIALand this country: US
added this owner: FISHER LAURENT BERNARDand this country: FR
added this owner: CACHET NATHALIE MICHELEand this country: FR
added this owner: BARZU-LE-ROUX SIMONAand this country: FR
12307
added this owner: ELIXIRGENand this country: US
12308
no country listed for: STANFORD UNIVERSITY
12309
matching owner countries: VERENIUM
matching owner countries: SYNGENTA PARTICIPATIONS AG
12310
matching owner countries: AMGEN
12311
matching owner countries: MEDICAL & BIOLOGICAL LABORATORIES CO.
matching owner countries: RIKEN
12312
this owner already in manual decisions:UNILEVER
this owner already in manual decisions:UNILEVER
this owner already in manual decisions:UNILEVER
12313
no country listed for: OPKO
12314
no country listed for: BATTELLE VENTURES
no country listed for: CORBION
no country listed for: CORBION
12315
no country listed for: NOVO NORDISK
12316
added this owner: METHEXIS GENOMICS N.Vand thi

In [16]:
len(owner_country_dict.keys())

6443

In [17]:
len(manual_decisions)

70

In [21]:
for i in manual_decisions: 
    if i not in add_affiliation_dict.keys(): 
        print(i)

GOEDEGEBUUR FRITS
SAGAWA HIROAKI
SANDGREN MATS
LI MING
LI JUN
LIU QING
MOCK THOMAS
FALCIATORE ANGELA
BOWLER CHRIS
YU LONG
LIU DONG
TONON THIERRY


In [ ]:
# write dictionary for country headquarters if they were not correct or if they had multiple registered

In [20]:
add_affiliation_dict = {'HOWARD HUGHES MEDICAL INSTITUTE': 'US',
 'individuals':np.nan,
 'CORBION': 'NL',
 'ABBVIE':'US',
 'PSOMAGEN': 'US',
 'EVONIK': 'DE',
 'BIOCERES':'AR',
 'NUFARM': 'AU',
 'ALTRUBIO':'US',
 'PHAIRSON MEDICAL':'GB',
 'STRYKER':'US',
 'EMERGENT BIOSOLUTIONS':'US',
 'CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE':'FR',
 'THERMO FISHER SCIENTIFIC':'US',
 'ASTRAZENECA': 'GB',
 'AGILENT TECHNOLOGIES': 'US',
 'ILLUMINA':'US',
 'BECTON DICKINSON AND COMPANY': 'US',
 'LAWRENCE LIVERMORE NATIONAL SECURITY':'US',
 'ADC THERAPEUTICS': 'CH',
 'LONZA':'CH',
 'ZEALAND PHARMA':'DK',
 'UNIVERSITY OF OTTOWA': 'CA',
 'ELANCO':'US',
 'CTC BIO':'KR',
 'SNIPR BIOME': 'DK',
 'MILLENNIUM PHARMACEUTICALS': 'US',
 'ARMISTICE CAPITAL MASTER FUND': 'US',
 'MESOBLAST INTERNATIONAL SARL': 'AU',
 'AB ENZYMES': 'DE',
 'GUARDANT HEALTH': 'US',
 'GRIFOLS':'ES',
 'FINCH THERAPEUTICS': 'US',
 'OPKO':'US',
 'JOHN HOPKINS UNIVERSITY': 'US',
 'TRANSWORLD': 'US',
 'ALPINE IMMUNE SCIENCES':'US',
 'DANA-FARBER CANCER INSTITUTE': 'US',
 'CRESTOVO': 'US',
 'UNIVERSITY OF PITTSBURGH': 'US',
 'U.S. SMOKELESS TOBACCO COMPANY': 'US',
 'BRISTOL-MYERS SQUIBB COMPANY': 'US',
 'BIO-RAD LABORATORIES': 'US',
 'MARICAL': 'US',
 'RAINDANCE TECHNOLOGIES': 'US',
 'ZYMERGEN': 'US',
 'INTARCIA THERAPEUTICS': 'US',
 'JIANGNAN UNIVERSITY':'CN',
 'IDEXX LABORATORIES': 'US',
 'YORKTOWN TECHNOLOGIES': 'US',
 'ROSETTA INPHARMATICS': 'US',
 'UNIVERSITY OF UTAH': 'US',
 'INSTITUT PASTEUR': 'BG',
 'UNIVERSITY LICENSE EQUITY HOLDING': 'US',
 'TWISTDX': 'GB',
 'NATIVE MICROBIALS': 'US',
 'VIRIDOS': 'US',
 'OSSIANIX': 'US',
 'ENZYMATICA': 'SE',
 'NINGBO UNIVERSITY':'CN',
 'LUCA TECHNOLOGIES': 'US',
 'GENERAL HOSPITAL CORPORATION': 'US',
 'DIVERGENCE': 'US',
 'SINGULAR GENOMICS SYSTEMS': 'US',
 'ARCA BIOPHARMA':'US',
 'JOHNSON & JOHNSON':'US',
 'CEDARS-SINAI MEDICAL CENTER': 'US',
 'ALPINE IMMUNE SCIENCES': 'US',
 'CARIBOU BIOSCIENCES': 'US',
 'THE JOHNS HOPKINS UNIVERSITY': 'US',
 'SANOFI': 'US',
 'ADVATECT DIAGNOSTICS': 'US',
 'LAWRENCE LIVERMORE NATIONAL SECURITY': 'US',
 'WASHINGTON UNIVERSITY': 'US',
 'THIRD WAVE TECHNOLOGIES': 'US',
 'QUARK PHARMACEUTICALS': 'IL',
 'UNIVERSITY OF IOWA RESEARCH FOUNDATION': 'US',
 'ARENA PHARMACEUTICALS': 'US',
 'ARISAPH PHARMACEUTICALS': 'US',
 'NEW YORK UNIVERSITY': 'US',
 'UNIVERSITY OF MARYLAND BALTIMORE': 'US',
 'EXXONMOBIL RESEARCH AND ENGINEERING COMPANY': 'US',
 'GENELUX CORPORATION': 'US',
 'THE CHEMOURS COMPANY FC': 'US',
 'RGH': np.nan,
 'CYTOMX THERAPEUTICS': 'US',
 'ROCHE': 'CH',
 'UNIVERSITY OF MARYLAND': 'US',
 'LUMEN BIOSCIENCE': 'US',
 'VANDERBILT UNIVERSITY': 'US',
 'NEW ENGLAND MEDICAL CENTER HOSPITALS': 'US',
 'HALOZYME THERAPEUTICS': 'US',
 'ENTERIS BIOPHARMA': 'US',
 'PENN STATE RESEARCH FOUNDATION THE': 'US',
 'FRED HUTCHINSON CANCER RESEARCH CENTER': 'US',
 'WOODS HOLE OCEANOGRAPHIC INSTITUTION': 'US',
 'NANOCYTE': 'IL',
 'JOULE UNLIMITED TECHNOLOGIES': 'US',
 'UNIVERSITY OF PENNSYLVANIA': 'US',
 'CHARLES RIVER LABORATORIES': 'US',
 'SMITH &NEPHEW': 'GB',
 'ASTERIAS BIOTHERAPEUITCS': 'US',
 'CAMBRIAN INNOVATION': 'US',
 'TRIAD NATIONAL SECURITY': 'US',
 'ALBERT EINSTEIN COLLEGE OF MEDICINE': 'US',
 "CHILDREN'S MEDICAL CENTER CORPORATION": 'US',
 'DOW':'US',
 'MACROGEN':'KR',
 'CORNELL RESEARCH FOUNDATION': 'US',
 'CANON':'JP',
 'HESKA CORPORATION': 'US',
 'LABORATORY CORPORATION OF AMERICA HOLDINGS': 'US',
 'ASURAGEN': 'US',
 'MIDPOINT FOOD AND AG CO-INVESTMENT FUND': 'US',
 'ROHM AND HAAS COMPANY': 'US',
 'ERAGEN BIOSCIENCES': 'US',
 'KALYPTO MEDICAL': 'US',
 'MEREO BIOPHARMA': 'GB',
 'PENNSYLVANIA STATE UNIVERSITY': 'US',
 'UNIGENE LABORATORIES': 'US',
 'GAUSSIA': np.nan,
 'BAYLOR COLLEGE OF MEDICINE': 'US',
 'BATTELLE VENTURES': 'US',
 'MIDDLELAND AG FUND L.P': 'GB',
 'ARIZONA STATE UNIVERSITY': 'US',
 'THE BROAD INSTITUTE': 'US',
 'J. CRAIG VENTER INSTITUTE': 'US',
 'LUMINEX CORPORATION': 'US',
 'MALLINCKRODT PHARMACEUTICALS': 'GB',
 'AMRYT': 'GB',
 'INNOVATION VALLEY PARTNERS': 'US',
 'UCB': 'BE',
 'BRAEMAR ENERGY VENTURES II': 'US',
 'RUTGERS THE STATE UNIVERSITY OF NEW JERSEY': 'US',
 'QIAGEN':'NL',
 'DOVETAIL GENOMICS': 'US',
 'THE BANK OF NEW YORK MELLON AS COLLATERAL AGENT': 'US',
 'TETRA HOLDING': 'DE',
 'REALTIME LABORATORIES': 'US',
 'MYCODART': 'US',
 'TUMOR BIOLOGY INVESTMENT GROUP': np.nan,
 'CELEXION': 'US',
 'FRAUENHOFER SOCIETY':'DE',
 'THE INVENTION SCIENCE FUND I': 'US',
 'UNIVERSITY OF NEW HAMPSHIRE': 'US',
 'UNIVERSITY OF MARYLAND': 'US',
 'BIOGEN': 'US',
 'CARNEGIE INSTITUTION OF WASHINGTON': 'US',
 'E. R. SQUIBB & SONS': 'US',
 'PROCTER & GAMBLE COMPANY THE': 'US',
 'UNIVERSITY OF CHICAGO': 'US',
 'NOVO NORDISK': 'DK',
 'SYNTIRON': 'US',
 'ALGENOL BIOTECH': 'US',
 'ALTRUBIO': 'US',
 'HESTZIG': 'US',
 'MARINE BIOLOGICAL LABORATORY': 'US',
 'VIRGINIA TECH INTELLECTUAL PROPERTIES': 'US',
 'BETH ISRAEL DEACONESS MEDICAL CENTER': 'US',
 'NUTECH VENTURES': 'US',
 'VERTEX PHARMACEUCTICALS': 'US',
 'UNITED PET GROUP': 'US',
 'BAXTER': 'US',
 'TAKEDA': 'JP',
 'UNIVERSITY OF TENNESSEE': 'US',
 'LABCORP':'US',
 'NOVOZYMES': 'DK',
 'NOVARTIS': 'CH',
 'XOMA': 'US',
 'DU PONT': 'US',
 'DSM': 'NL',
 'UNILEVER': 'GB',
 'MERCK': 'DE',
 '4D PHARMA': 'GB',
 'STRATAGENE':'US',
 'PFIZER':'US',
 'GINKGO BIOWORKS':'US',
 'EW GROUP':'DE',
 'BASF': 'DE',
 'SEEGENE': 'KR',
 'ALEXION': 'US',
 'JNC CORPORATION': 'JP',
 'SAMANA BUSINESS': 'IN',
 'BAYER': 'DE',
 'TRANZYME': 'US',
 'COLOMBIA UNIVERSITY': 'US',
 'EVROGEN':'RU',
 'INVISTA': 'US',
 'INTREXON': 'US',
 'GLAXOSMITHKLINE':'GB',
 'INSERM':'FR',
 'ALGENOL':'US',
 'VERTEX PHARMACEUTICALS':'US',
 'TAKARA':'JP',
 'KOREA INSTITUTE OF OCEAN SCIENCE & TECHNOLOGY':'KR',
 'ASTELLAS':'JP',
 'FVG':np.nan,
 'EVOLVA':'CH',
 'HOLOGIC':'US',
 'NEC':'JP',
 'POSTECH ACADEMY-INDUSTRY FOUNDATION':'KR',
'INDIGO AG': 'US'}

In [22]:
owner_country_dict_completed = owner_country_dict.copy()

In [23]:
len(add_affiliation_dict)

198

In [24]:
# add everything together and assign to country column
for k,v in add_affiliation_dict.items(): 
    owner_country_dict_completed[k] = v


In [25]:
df_patents_ed['edited_company_country'] = df_patents_ed['edited_company_name'].map(owner_country_dict_completed)

<ipython-input-25-c2756a159135>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_patents_ed['edited_company_country'] = df_patents_ed['edited_company_name'].map(owner_country_dict_completed)


In [26]:
#add value whether companies were edited/merged or not... (for transparency)

In [27]:
manually_edited_companies = list(set([x for x in edited_names_owners_noincs if x not in unedited_names_owners_noincs])) #add if I either merged names, or manually changed their countries

In [28]:
len(manually_edited_companies)

409

In [29]:
# add as column

In [30]:
manually_edited = []
for i in df_patents_ed['edited_company_name']:
    if i in manually_edited_companies:
        manually_edited.append('manually edited')
    if i not in manually_edited_companies:
        manually_edited.append('unedited')

In [31]:
df_patents_ed['edited'] = manually_edited

<ipython-input-31-67c07579070d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_patents_ed['edited'] = manually_edited


In [32]:
# change KP to KR because this is just a mistake in the data
df_patents_ed_noKP = df_patents_ed.replace({'edited_company_country':{'KP':'KR'}})

In [37]:
# export data
df_patents_ed_noKP.to_csv('PATH\data\CoP.csv')